In [1]:
cd "/Users/sylviathsu/Documents/COS243/LocalAILibrarian"

/Users/sylviathsu/Documents/COS243/LocalAILibrarian


In [2]:
!pip install -q \
  llama-index \
  EbookLib \
  html2text \
  gradio \
  llama-index-embeddings-huggingface \
  llama-index-llms-ollama

In [3]:
import os
import logging
import gradio as gr
from llama_index.core import (
    SimpleDirectoryReader, 
    VectorStoreIndex, 
    StorageContext, 
    load_index_from_storage,
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core.node_parser import SentenceSplitter
from pathlib import Path

/opt/anaconda3/envs/gradio/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import os
import logging
import gradio as gr
from pathlib import Path
import shutil
from typing import List, Tuple
import json
from datetime import datetime

from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core.node_parser import SentenceSplitter

def setup_logging():
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    )
    return logging.getLogger(__name__)

logger = setup_logging()

# [Previous functions remain unchanged: process_documents, generate_embeddings, configure_query_engine]

def save_conversation_history(history: List[Tuple[str, str]], file_format: str = "txt") -> str:
    """Save conversation history to a file"""
    try:
        # Create exports directory if it doesn't exist
        export_dir = Path("exports")
        export_dir.mkdir(exist_ok=True)
        
        # Generate timestamp for filename
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        if file_format == "json":
            # Save as JSON
            filename = export_dir / f"conversation_{timestamp}.json"
            conversation_data = {
                "timestamp": timestamp,
                "messages": [{"question": q, "answer": a} for q, a in history]
            }
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(conversation_data, f, indent=2, ensure_ascii=False)
        else:
            # Save as plain text
            filename = export_dir / f"conversation_{timestamp}.txt"
            with open(filename, 'w', encoding='utf-8') as f:
                for q, a in history:
                    f.write(f"Q: {q}\n")
                    f.write(f"A: {a}\n")
                    f.write("-" * 80 + "\n")
        
        return f"Conversation saved to {filename}"
    except Exception as e:
        logger.error(f"Error saving conversation: {e}")
        return f"Error saving conversation: {str(e)}"

def create_gradio_interface():
    """Create and configure the Gradio interface"""
    # Initialize query engine at the start
    global_query_engine = configure_query_engine() if Path("storage").exists() else None
    
    def handle_file_upload(files, progress=gr.Progress()):
        """Handle file upload and index update"""
        try:
            nonlocal global_query_engine
            
            # Create library directory if it doesn't exist
            Path("library").mkdir(exist_ok=True)
            
            # Copy uploaded files to library directory
            progress(0, desc="Copying files...")
            for file in files:
                shutil.copy2(file.name, "library")
            
            # Process documents and update index
            progress(0.3, desc="Processing documents...")
            documents = process_documents("library")
            
            progress(0.6, desc="Generating embeddings...")
            generate_embeddings(documents)
            
            progress(0.9, desc="Configuring query engine...")
            global_query_engine = configure_query_engine()
            
            progress(1.0, desc="Done!")
            return "Files uploaded and indexed successfully!"
        except Exception as e:
            logger.error(f"Upload error: {e}")
            return f"Error processing files: {str(e)}"

    def query_docs(query: str, history: List[Tuple[str, str]]) -> Tuple[str, List]:
        nonlocal global_query_engine
        
        if global_query_engine is None:
            error_msg = "Please upload documents first or wait for indexing to complete."
            history.append((query, error_msg))
            return "\n".join([f"Q: {q}\nA: {a}" for q, a in history]), history
        
        try:
            response = global_query_engine.query(query)
            
            # Extract source information
            sources = []
            if hasattr(response, 'source_nodes'):
                sources = [
                    f"- {node.node.metadata.get('file_name', 'Unknown')}"
                    for node in response.source_nodes
                ]
            
            source_text = "\nSources:\n" + "\n".join(sources) if sources else ""
            full_response = str(response) + source_text
            
            history.append((query, full_response))
            return "\n".join([f"Q: {q}\nA: {a}" for q, a in history]), history
            
        except Exception as e:
            logger.error(f"Query processing error: {e}")
            error_msg = f"Error processing query: {str(e)}"
            history.append((query, error_msg))
            return "\n".join([f"Q: {q}\nA: {a}" for q, a in history]), history

    with gr.Blocks(title="Local AI Librarian") as app:
        gr.Markdown("# Local AI Librarian")
        gr.Markdown("Upload documents and search through them using natural language queries.")
        
        # Document upload section
        with gr.Row():
            with gr.Column():
                file_output = gr.Textbox(label="Upload Status")
                upload_button = gr.File(
                    file_count="multiple",
                    label="Upload Documents",
                    file_types=[".txt", ".pdf", ".epub"]
                )
        
        # Query section
        with gr.Row():
            with gr.Column(scale=2):
                query_input = gr.Textbox(
                    label="Enter your query",
                    placeholder="What would you like to know about your documents?"
                )
                examples = gr.Examples(
                    examples=[
                        "What are the main themes in the documents?",
                        "Summarize the key points about...",
                        "Find relevant passages about..."
                    ],
                    inputs=query_input
                )
                
        with gr.Row():
            submit_btn = gr.Button("Search")
            clear_btn = gr.Button("Clear History")
            
        # Export controls
        with gr.Row():
            export_format = gr.Radio(
                choices=["txt", "json"],
                value="txt",
                label="Export Format"
            )
            export_btn = gr.Button("Export Conversation")
            export_status = gr.Textbox(label="Export Status")
            
        chat_history = gr.State([])
        output = gr.Textbox(
            label="Results",
            lines=20,
            autoscroll=False
        )
        
        # Set up event handlers
        upload_button.upload(
            handle_file_upload,
            inputs=[upload_button],
            outputs=[file_output]
        )
        
        submit_btn.click(
            query_docs,
            inputs=[query_input, chat_history],
            outputs=[output, chat_history]
        )
        
        clear_btn.click(
            lambda: ([], []),
            outputs=[output, chat_history]
        )
        
        export_btn.click(
            save_conversation_history,
            inputs=[chat_history, export_format],
            outputs=[export_status]
        )
        
    return app

if __name__ == "__main__":
    try:
        logger.info("Starting application...")
        
        # Launch Gradio interface
        logger.info("Launching Gradio interface...")
        app = create_gradio_interface()
        app.launch(share=False)
        
    except Exception as e:
        logger.error(f"Application error: {e}")
        raise

INFO:__main__:Starting application...
INFO:__main__:Launching Gradio interface...
INFO:llama_index.core.indices.loading:Loading all indices.
INFO:httpx:HTTP Request: GET http://127.0.0.1:7867/startup-events "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD http://127.0.0.1:7867/ "HTTP/1.1 200 OK"


Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
Generating embeddings: 100%|██████████| 662/662 [02:28<00:00,  4.46it/s]
INFO:llama_index.core.indices.loading:Loading all indices.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
ERROR:__main__:Query processing error: shapes (1536,) and (384,) not aligned: 1536 (dim 0) != 384 (dim 0)
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
ERROR:__main__:Query processing error: shapes (1536,) and (384,) not aligned: 1536 (dim 0) != 384 (dim 0)
